In [1]:
""" Created on June 12, 2021 // @author: Sarah Shi and Henry Towbin """

' Created on June 12, 2021 // @author: Sarah Shi and Henry Towbin '

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import mc3
import MC3_BASELINES as baselines

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rc, cm

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
rc('font',**{'size': 12}) # 'family':'Avenir'
plt.rcParams['pdf.fonttype'] = 42

In [3]:
path_parent = os.path.dirname(os.getcwd())
path_beg = os.getcwd() + '/'
path_input = os.getcwd() + '/Inputs/'
output_dir = ["FIGURES", "PLOTFILES", "NPZFILES", "LOGFILES", "FINALDATA"] 

# Create output directories if not in existence 
for ii in range(len(output_dir)):
    if not os.path.exists(path_beg + output_dir[ii]):
       os.makedirs(path_beg + output_dir[ii], exist_ok=True)

# Change paths to direct to folder with SampleSpectra -- last bit should be whatever your folder with spectra is called. 
PATHS = [path_input+'SampleSpectra/Fuego/', path_input+'SampleSpectra/Standards/', path_input+'SampleSpectra/Fuego1974RH/', path_input+'SampleSpectra/SIMS/', path_input+'SampleSpectra/SIMSStd_2/']

# Put ChemThick file in Inputs. Direct to what your ChemThick file is called. 
CHEMTHICK_PATH = [path_input+'FuegoChemThick.csv', path_input+'StandardChemThick.csv', path_input+'DanRHChemThick.csv', path_input+'SIMSChemThick.csv', path_input+'SIMSChemThick_2.csv']

# Change last value in list to be what you want your output directory to be called. 
INPUT_PATHS = [[path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'FUEGO_F'],
                [path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'STD_F'], 
                [path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'FRH_F'],
                [path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'SIMS_F'],
                [path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'SIMS2_F']]

# Change to be what you want the prefix of your output files to be. 
OUTPUT_PATH = ['F18', 'STD', 'FRH', 'SIMSSTD', 'SIMSSTD2']

In [4]:
REF_PATH = path_input + '/ReflectanceSpectra/FuegoOl/'
REF_FILES = glob.glob(REF_PATH + "*")
REF_FILES.sort()

REF_DFS_FILES, REF_DFS_DICT = baselines.Load_SampleCSV(REF_FILES, wn_high = 2800, wn_low = 2000)

# Use DHZ parameterization of olivine reflectance index. 
n_ol = baselines.ReflectanceIndex(0.72)

REF_FUEGO = baselines.ThicknessProcessing(REF_DFS_DICT, n = n_ol, wn_high = 2700, wn_low = 2100, remove_baseline = True, plotting = False, phaseol = True)

In [5]:
REF_PATH = path_input + '/ReflectanceSpectra/rf_ND70/'
REF_FILES = glob.glob(REF_PATH + "*")
REF_FILES.sort()

REF_DFS_FILES, REF_DFS_DICT = baselines.Load_SampleCSV(REF_FILES, wn_high = 2850, wn_low = 1700)

# n=1.546 in the range of 2000-2700 cm^-1 following Nichols and Wysoczanski, 2007 for basaltic glass
n_gl = 1.546

REF_FUEGO = baselines.ThicknessProcessing(REF_DFS_DICT, n = n_gl, wn_high = 2850, wn_low = 1700, remove_baseline = True, plotting = False, phaseol = False)

In [6]:
simsno = 3 

PATH = PATHS[simsno]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[simsno])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[simsno])


DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[simsno] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[simsno] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[simsno] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[simsno] + '_DensityEpsilon.csv')
MEGA_SPREADSHEET


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 2.02473245e+00  1.87020955e+00  2.86142095e-01 -2.54504598e-02
 -3.61383535e-01  1.43166755e+03  3.52028432e+01  3.49692362e-01
  1.52368725e+03  3.11145163e+01  3.71188423e-01  3.53164253e-01
 -2.03646625e-01  4.14774744e-01  3.03192461e-05  1.12919806e+00]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Sat Mar  4 18:26:28 2023)

[:         ]  10.0% completed  (Sat Mar  4 18:26:40 2023)
Out-of-bound Trials:
[  4 436  49 853 315 217 292   0 359  98   0   2  24 243   0   0]
Best Parameters: (chisq=77.7379)
[ 2.02473245e+00  1.87020955e+00  2.86142095e-01 -2.54504598e-02
 -3.61383535e-01  1.43166755e+03  3.5202843

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,CO2_1430_BP,CO2_1430_STD,H2Om_5200_M,H2Om_5200_STD,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500
ND70_02_01_06032022_150x150_sp1,1.879066,0.190595,2.821279,0.218038,*,0.664208,0.121441,1328.070603,42.707246,1353.240783,...,1274.872641,59.201485,0.719281,0.211139,1.19503,0.356532,9.035935,9.666112,-,-
ND70_02_01_06032022_150x150_sp2,1.944956,0.197358,2.82231,0.217844,*,0.68487,0.125138,1341.091665,43.089322,1366.770053,...,1289.076959,59.952151,0.545321,0.157263,1.240988,0.369984,8.640499,14.06714,-,-
ND70_02_01_06032022_150x150_sp3,1.783725,0.177168,2.818011,0.21758,*,0.660478,0.120724,1364.048723,43.722794,1389.331414,...,1307.13407,60.394884,0.70833,0.20718,1.102565,0.328987,6.542633,12.275726,-,-
ND70_03_01_06032022_150x150_sp1,2.596221,0.251282,2.530655,0.197525,*,1.116795,0.208069,1799.448095,57.016656,1863.107598,...,1738.457326,78.679417,1.013587,0.30432,1.481352,0.456824,9.218763,11.912369,-,-
ND70_03_01_06032022_150x150_sp2,2.781793,0.268289,5.60941,0.437564,*,1.163778,0.216882,2021.45994,63.905855,2024.505565,...,1893.657718,85.40179,1.013036,0.303797,1.532174,0.472564,11.620757,12.624295,-,-
ND70_03_01_06032022_150x150_sp3,2.658923,0.254330,5.644396,0.439773,*,1.130408,0.210837,2004.067515,63.409442,1997.885561,...,1879.549433,84.938435,0.980969,0.294109,1.441811,0.44536,9.356311,13.720353,-,-
ND70_04_02_06032022_150x150_sp1,2.771658,0.244441,5.491872,0.425807,*,1.264966,0.228288,3077.092551,96.016647,3090.144182,...,2881.813342,127.608106,0.914975,0.257268,1.424622,0.416166,7.457396,11.450836,-,-
ND70_04_02_06032022_150x150_sp2,3.027419,0.265296,5.496119,0.425916,*,1.413128,0.255266,3376.824397,105.063924,3401.700407,...,3170.944128,139.828428,1.290134,0.362503,1.533154,0.448227,8.750288,13.842134,-,-
ND70_04_02_06032022_150x150_sp3,3.000831,0.258904,3.145139,0.243946,*,1.479921,0.266528,3352.98225,104.17161,3473.982373,...,3221.195003,141.957002,1.2647,0.354559,1.516082,0.443489,10.810417,13.442954,-,-
ND70_05_02_06032022_150x150_sp1,4.168685,0.357250,4.267484,0.323647,*,2.481155,0.469657,8090.721851,229.571631,8607.722823,...,7555.720014,303.639651,2.053071,0.639611,1.682892,0.537863,10.696872,12.864924,-,-


In [7]:
simsno2 = 4

PATH = PATHS[simsno2]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[simsno2])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[simsno2])
DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[simsno2] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[simsno2] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[simsno2] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[simsno2] + '_DensityEpsilon.csv')
MEGA_SPREADSHEET


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 6.41441089e-01  1.16020852e+00  1.45840638e-01  2.49613052e-01
 -1.61006029e-01  1.43146891e+03  3.69854098e+01  1.10837392e-01
  1.52467183e+03  3.15926962e+01  1.14182783e-01  1.02695681e-01
 -5.48357532e-02  1.23509348e-01  2.57413859e-04  1.70104625e-01]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Sat Mar  4 18:47:25 2023)

[:         ]  10.0% completed  (Sat Mar  4 18:47:30 2023)
Out-of-bound Trials:
[ 109  328    5 1227   56  219 6124    4  624  521    4    5    3    3
    0    0]
Best Parameters: (chisq=18.4540)
[ 6.41441089e-01  1.16020852e+00  1.45840638e-01  2.49613052e-01
 -1.61006029e-01  1.431468

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,CO2_1430_BP,CO2_1430_STD,H2Om_5200_M,H2Om_5200_STD,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500
ND70_02-01_30June2022_150x150_sp1,3.853961,0.608520,3.853961,0.60852,-,1.011393,0.239602,2147.891906,230.224535,2179.824764,...,2115.959048,326.435805,1.134248,0.384299,1.09816,0.372949,7.753115,11.927947,-,-
ND70_02-01_30June2022_150x150_sp2,3.727927,0.588352,3.727927,0.588352,-,0.986785,0.234012,1838.345294,202.029361,1898.080212,...,1778.610376,282.701086,0.299566,0.099066,2.93305,1.039902,3.872114,13.553811,*,-
ND70_02-01_30June2022_150x150_sp3,3.427590,0.481919,3.42759,0.481919,-,0.879875,0.197484,1742.278091,167.289747,1799.868436,...,1684.687747,235.229118,2.187381,0.743691,0.729587,0.239777,19.547807,5.259082,-,-
ND70_02-01_30June2022_150x150_sp4,4.005539,0.633002,4.005539,0.633002,-,1.075633,0.254483,1963.002461,211.747263,2042.541017,...,1883.463905,293.144877,0.893176,0.311416,4.9633,1.720563,2.633839,3.518766,*,*
ND70_03-01_30June2022_150x150_sp1,3.329736,0.428524,3.329736,0.428524,-,1.166663,0.253315,1901.583145,158.74879,1934.192518,...,1868.973772,225.331185,1.231806,0.401537,2.071944,0.700941,17.132012,9.279202,-,-
ND70_03-01_30June2022_150x150_sp2,3.785508,0.531605,3.785508,0.531605,-,1.325335,0.299734,2190.241189,202.92346,2233.297533,...,2147.184844,285.80625,0.934881,0.311934,2.566596,0.900282,6.871001,9.459134,-,-
ND70_03-01_30June2022_150x150_sp3,4.138584,0.583034,4.138584,0.583034,-,1.461392,0.32923,2365.073434,217.67113,2418.321976,...,2311.824892,305.512099,0.75757,0.250522,3.183175,1.109006,3.13071,11.721556,*,-
ND70_03-01_30June2022_150x150_sp4,5.023326,0.792742,5.023326,0.792742,-,1.622611,0.37743,2823.63674,294.582087,2940.310167,...,2706.963312,403.064,2.109863,0.683305,4.640547,1.549954,1.930331,3.579585,*,*
ND70_0503_29June2022_100x60_256s_sp1,5.223795,0.736479,5.223795,0.736479,-,1.845726,0.421578,1418.458749,141.367445,1446.817785,...,1390.099713,202.582487,3.196575,1.120459,2.006656,0.701324,7.591094,7.324789,-,-
ND70_0503_29June2022_95x80_256s_sp2,4.503154,0.579459,4.503154,0.579459,-,1.604693,0.353211,1001.383025,94.223538,1075.565325,...,927.200725,128.940783,1.792367,0.600214,1.755768,0.601238,8.795233,6.883495,-,-


In [8]:
stdno = 1

PATH = PATHS[stdno]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[stdno])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[stdno])
DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_DensityEpsilon.csv')

def STD_DF_MOD(MEGA_SPREADSHEET):
    STD_VAL = pd.DataFrame(index = MEGA_SPREADSHEET.index, columns = ['H2O_EXP', 'H2O_EXP_STD', 'CO2_EXP', 'CO2_EXP_STD'])

    for j in MEGA_SPREADSHEET.index: 
            
        if '21ALV1846' in j: 
            H2O_EXP= 1.89
            H2O_EXP_STD = 0.19
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif '23WOK5-4' in j: 
            H2O_EXP = 1.6
            H2O_EXP_STD = 0.16
            CO2_EXP = 64	
            CO2_EXP_STD = 6.4
        elif 'ALV1833-11' in j: 
            H2O_EXP = 1.2
            H2O_EXP_STD = 0.12
            CO2_EXP = 102
            CO2_EXP_STD = 10.2
        elif 'CD33_12-2-2' in j: 
            H2O_EXP = 0.27
            H2O_EXP_STD = 0.03
            CO2_EXP = 170
            CO2_EXP_STD = 17
        elif 'CD33_22-1-1' in j: 
            H2O_EXP = 0.49
            H2O_EXP_STD = 0.05
            CO2_EXP = 109
            CO2_EXP_STD = 10.9
        elif 'ETFSR_Ol8' in j: 
            H2O_EXP = 4.16
            H2O_EXP_STD = 0.42
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'Fiege63' in j: 
            H2O_EXP = 3.10
            H2O_EXP_STD = 0.31
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'Fiege73' in j: 
            H2O_EXP = 4.47
            H2O_EXP_STD = 0.45
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'STD_C1' in j: 
            H2O_EXP = 3.26
            H2O_EXP_STD = 0.33
            CO2_EXP = 169
            CO2_EXP_STD = 16.9
        elif 'STD_CN92C_OL2' in j: 
            H2O_EXP = 4.55
            H2O_EXP_STD = 0.46
            CO2_EXP = 270
            CO2_EXP_STD = 27
        elif 'STD_D1010' in j: 
            H2O_EXP = 1.13
            H2O_EXP_STD = 0.11
            CO2_EXP = 139
            CO2_EXP_STD = 13.9
        elif 'STD_ETFS' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_127-7' in j: 
            H2O_EXP = 3.98
            H2O_EXP_STD = 0.39
            CO2_EXP = 439
            CO2_EXP_STD = 43.9
        elif 'VF74_131-1' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_131-9' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_132-1' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_132-2' in j: 
            H2O_EXP = 3.91
            H2O_EXP_STD = 0.39
            CO2_EXP = 198
            CO2_EXP_STD = 19.8
        elif 'VF74_134D-15' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_136-3' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan

        STD_VAL.loc[j] = pd.Series({'H2O_EXP':H2O_EXP,'H2O_EXP_STD':H2O_EXP_STD,'CO2_EXP':CO2_EXP,'CO2_EXP_STD':CO2_EXP_STD})

    MEGA_SPREADSHEET_STD = pd.concat([MEGA_SPREADSHEET, STD_VAL], axis = 1)

    return MEGA_SPREADSHEET_STD

MEGA_SPREADSHEET_STD = STD_DF_MOD(MEGA_SPREADSHEET)

MEGA_SPREADSHEET_STD.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_H2OCO2_FwSTD.csv')

MEGA_SPREADSHEET_STD


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 1.35552143e+00  3.52047737e+00  1.00249803e-01  4.67072495e-01
  1.03168631e-01  1.42166756e+03  3.99999552e+01  1.23059552e-18
  1.52922607e+03  2.56001143e+01  8.36005679e-18  1.75529655e-01
 -1.84764506e-02  1.04670588e-02  1.66024932e-04  7.22002617e-01]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Sat Mar  4 19:05:26 2023)

[:         ]  10.0% completed  (Sat Mar  4 19:05:33 2023)
Out-of-bound Trials:
[    2    52     1   624    13 15003 13308 11625 14805 13598 11380     1
     1     1     0     0]
Best Parameters: (chisq=48.5469)
[ 1.35552143e+00  3.52047737e+00  1.00249803e-01  4.67072495e-01
  1.031686

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500,H2O_EXP,H2O_EXP_STD,CO2_EXP,CO2_EXP_STD
21ALV1846-9_44x44_s1_EP_103um,1.387128,0.108216,1.387128,0.108216,-,0.347536,0.058329,0.0,3.982073,0.0,...,0.83727,0.214254,10.02839,7.156581,-,-,1.89,0.19,NaN,NaN
21ALV1846-9_44x44_s2_EP_103um,1.490365,0.116293,1.490365,0.116293,-,0.354715,0.059455,0.0,4.017368,0.0,...,0.803276,0.205768,7.212676,10.331606,-,-,1.89,0.19,NaN,NaN
21ALV1846-9_44x44_s3_EP_103um,1.398400,0.109211,1.3984,0.109211,-,0.347982,0.058423,0.0,4.029611,0.0,...,0.832492,0.212694,10.619909,8.536247,-,-,1.89,0.19,NaN,NaN
23WOK5-4_44x44_s1_EP_147um,1.043671,0.078271,1.043671,0.078271,-,0.211256,0.035565,30.917425,8.829732,31.805809,...,0.540746,0.139793,6.536997,9.246675,-,-,1.6,0.16,64.0,6.4
23WOK5-4_44x44_s2_EP_147um,0.974799,0.073185,0.974799,0.073185,-,0.208961,0.035203,40.045603,9.032437,51.730308,...,0.55573,0.143407,7.202544,10.437003,-,-,1.6,0.16,64.0,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VF74_134D-15_101220_256s_15x25_b,6.318766,0.932922,6.318766,0.932922,-,4.423068,0.860016,1241.126131,140.745858,1156.300521,...,3.194434,0.798485,4.872235,4.565704,-,-,NaN,NaN,NaN,NaN
VF74_134D-15_101220_256s_15x25_c,6.333227,0.933222,6.333227,0.933222,-,4.436521,0.863054,1260.808701,143.000959,1161.463704,...,3.67351,0.929255,5.839047,7.742985,-,-,NaN,NaN,NaN,NaN
VF74_136-3_101220_256s_20x30_a,4.066974,0.490267,4.066974,0.490267,-,1.986081,0.351699,598.329298,82.092156,584.73856,...,1.528126,0.362124,5.896663,5.254649,-,-,NaN,NaN,NaN,NaN
VF74_136-3_101220_256s_20x30_b,4.072526,0.490356,4.072526,0.490356,-,1.978682,0.349424,609.976438,82.920486,592.891491,...,1.418366,0.340154,7.237378,6.400376,-,-,NaN,NaN,NaN,NaN


In [9]:
fuegono = 0 

PATH = PATHS[fuegono]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[fuegono])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[fuegono])
DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[fuegono] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[fuegono] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[fuegono] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[fuegono] + '_DensityEpsilon.csv')
MEGA_SPREADSHEET


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 5.23364283e-01 -1.06184898e+00  5.08079324e-01 -1.87672707e-01
  2.01907538e-01  1.43095396e+03  2.35623242e+01  2.26234604e-02
  1.51695258e+03  4.00000000e+01  1.97532294e-02  2.47809786e-01
  4.59210479e-02 -1.28999055e-01 -3.66655504e-04  8.52450022e-01]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Sat Mar  4 23:58:01 2023)

[:         ]  10.0% completed  (Sat Mar  4 23:58:06 2023)
Out-of-bound Trials:
[  167   751    28   626    88  2825 10481    78  3806 12214   126     1
     2     0     0     0]
Best Parameters: (chisq=41.4400)
[ 5.23364283e-01 -1.06184898e+00  5.08079324e-01 -1.87672707e-01
  2.019075

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,CO2_1430_BP,CO2_1430_STD,H2Om_5200_M,H2Om_5200_STD,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500
AC4_EUH102_030920_256s_15x20_a,2.711240,0.266554,2.71124,0.266554,-,1.03292,0.172404,253.896272,37.892836,236.699532,...,271.093013,55.004148,1.266691,0.279764,1.249587,0.29939,3.628526,4.414297,*,-
AC4_EUH102_030920_256s_15x20_b,2.716639,0.267521,2.716639,0.267521,-,1.036414,0.173153,282.499543,38.099512,279.423521,...,285.575566,54.548427,1.28776,0.282712,1.06921,0.256798,3.300287,4.74223,*,-
AC4_EUH102_030920_256s_15x20_c,2.740825,0.269765,2.740825,0.269765,-,1.034883,0.172341,256.496656,37.128673,262.443202,...,250.550111,53.444213,1.054834,0.274014,1.837795,0.436475,2.793253,4.524856,*,-
AC4_EUH33_030920_256s_20x20_a,1.386910,0.130896,1.38691,0.130896,-,0.237801,0.039752,96.556071,31.103698,55.296699,...,137.815442,52.881644,0.409759,0.082262,1.239329,0.290226,1.737557,5.13387,*,-
AC4_EUH33_030920_256s_20x20_b,1.391496,0.131182,1.391496,0.131182,-,0.239994,0.040008,89.729354,30.373825,47.089942,...,132.368767,52.445856,0.405542,0.081421,1.131029,0.249665,1.289942,5.637654,*,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC4_RH3_OL2_022120_256s_30x30_b,3.134099,0.289819,3.134099,0.289819,-,1.341083,0.228802,693.380617,65.337557,700.082797,...,686.678438,97.786095,1.160055,0.273457,1.29819,0.332925,4.11666,6.231784,-,-
AC4_RH3_OL2_022120_256s_30x30_c,3.168745,0.293481,3.168745,0.293481,-,1.341095,0.22855,700.492836,65.15115,705.180333,...,695.805338,97.420098,1.098759,0.258141,1.359358,0.348166,4.649161,5.146355,-,-
AC4_RH3_OL3_022120_256s_20x20_a,2.354550,0.189186,2.35455,0.189186,-,1.220063,0.179136,1180.853185,61.277291,1210.110221,...,1151.596149,86.337099,1.258648,0.238209,1.314868,0.275151,4.465166,8.194618,-,-
AC4_RH3_OL3_022120_256s_20x20_b,2.589110,0.168727,2.378323,0.191263,*,1.226812,0.180105,1165.835608,60.489605,1199.364153,...,1137.165288,85.098563,1.128066,0.21884,1.367693,0.286267,3.546042,6.591443,*,-


In [10]:
fuegorhno = 2 

PATH = PATHS[fuegorhno]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[fuegorhno])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[fuegorhno])
DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[fuegorhno] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[fuegorhno] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[fuegorhno] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[fuegorhno] + '_DensityEpsilon.csv')
MEGA_SPREADSHEET


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 7.37787560e-01 -5.60018680e-01  4.62635951e-01  4.20236336e-02
  1.56984539e-01  1.42184213e+03  2.25000000e+01  1.04128798e-03
  1.53500000e+03  2.25000000e+01  5.24596018e-04  3.83359741e-01
 -1.33635200e-01  9.36261562e-03 -1.38172479e-04  7.44671901e-01]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Sun Mar  5 01:26:31 2023)

[:         ]  10.0% completed  (Sun Mar  5 01:26:36 2023)
Out-of-bound Trials:
[   36   200     8   168    46 14364 13628  8490 14773 13333  9401     0
     0     0     0     0]
Best Parameters: (chisq=9.5069)
[ 7.37787560e-01 -5.60018680e-01  4.62635951e-01  4.20236336e-02
  1.5698453

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,CO2_1430_BP,CO2_1430_STD,H2Om_5200_M,H2Om_5200_STD,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500
FI45MI02a,3.184887,0.285928,3.543211,0.317689,*,1.440164,0.265713,6.045002,12.926605,4.033839,...,8.0069,19.341279,1.482843,0.419299,1.731744,0.50352,6.145607,4.434907,-,-
FI45MI02b,2.866311,0.247130,3.536424,0.317104,*,1.432953,0.264525,15.772235,14.303756,21.807649,...,9.495621,19.35996,1.699944,0.473265,1.411441,0.41258,8.076008,15.825501,-,-
FI45MI02c,3.109986,0.277351,3.483238,0.312245,*,1.440537,0.265975,13.911582,13.521833,17.963443,...,9.741449,18.990959,1.610133,0.449124,1.656229,0.483725,4.812446,7.790131,-,-
FI45MI08a,3.008273,0.272631,3.315637,0.275512,*,1.40112,0.259027,124.553823,12.044147,0.0,...,248.225573,23.567016,1.6628,0.480984,1.596501,0.478136,2.553545,5.505508,*,-
FI45MI08b,3.227003,0.300395,5.009617,0.417333,*,1.394106,0.25748,115.208401,11.849377,0.0,...,225.783848,22.832947,1.541003,0.456675,1.768012,0.529102,3.182398,7.39897,*,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FR06MI07b,4.141785,0.595559,4.141785,0.595559,-,1.824082,0.384488,2535.15086,251.933153,2607.404763,...,2462.896956,354.112748,1.2169,0.347809,3.438855,1.008938,0.931742,3.493894,*,*
FR06MI07c,4.033439,0.579924,4.033439,0.579924,-,1.855982,0.390758,2544.296263,252.452145,2683.394051,...,2405.198476,346.248221,2.753953,0.769204,3.079374,0.903725,1.627286,3.289291,*,*
FR06MI08a,3.691424,0.411819,3.691424,0.411819,-,1.696776,0.340977,2502.140463,173.936475,2516.062423,...,2488.218502,247.728588,1.183315,0.356912,0.762338,0.241124,3.235629,1.776139,*,*
FR06MI08b,3.637252,0.405776,3.637252,0.405776,-,1.679971,0.338059,2475.553763,172.120828,2491.387264,...,2459.720262,245.156902,1.401588,0.420314,0.980492,0.305185,3.504198,4.301674,*,-
